In [15]:
import PyPDF2
import os
import PyPDF2
import pandas as pd

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

from brave import Brave
import time
from tqdm import tqdm



# Search for Job centers using Brave Search API

In [14]:
jc_list = pd.read_excel('../Datasets/JobCenter.xlsx')
brave = Brave('BSAarKAc1psxHA5gp3fRLtkm5n40lGC')

In [16]:
searches = []

# Adding a delay to avoid sending too many requests per second, this got me errors.
request_delay = 1

for jc in jc_list['Region']:
    # Example Berlin Jobcenter
    query = jc + 'Jobcenter'
    num_results = 2

    try:
        # Perform the search request
        search_results = brave.search(q=query, count=num_results)
        searches.append(search_results)
        
    except Exception as error:
        # Handling errors
        print(f"An error occurred while searching for '{query}': {error}")

    # delay between requests
    time.sleep(request_delay)


An error occurred while searching for 'KyffhäuserkreisJobcenter': 1 validation error for WebSearchApiResponse
query.is_navigational
  Field required [type=missing, input_value={'original': 'Kyffhäuser...le': False, 'state': ''}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing


# Arranging the results of the search in a Dataframe for management ease

In [17]:
search_results_list = []
for search in searches:

    # Iterate over each web result in the search
    for web_result in search.web_results:

        # Extract info from Brave searches objects
        title = web_result['title']
        url = web_result['url']
        description = web_result['description']
        language = web_result['language']
        
        # Append the information to the list as a dictionary
        search_results_list.append({
            'Title': title,
            'URL': url,
            'Description': description,
            'Language': language,
        })

# Create a DataFrame from the list of dictionaries
search_results_df = pd.DataFrame(search_results_list)

search_results_df

,Title,URL,Description,Language
0,Willkommen beim Jobcenter Alb-Donau - jobcente...,https://www.jobcenter-alb-donau.de/,Hinweis: Unsere Informationen richten sich an ...,de
1,Jobcenter Alb-Donau - 89073 Ulm | Bundesagentu...,https://www.arbeitsagentur.de/vor-ort/jobcente...,Willkommen beim <strong>Jobcenter</strong> <st...,de
2,Jobcenter - Stadt Baden-Baden,https://www.baden-baden.de/buergerservice/serv...,<strong>Das Jobcenter ist eine gemeinsame Einr...,de
3,Jobcenter Baden - Baden - 76532 Baden-Baden | ...,https://www.arbeitsagentur.de/vor-ort/jobcente...,Willkommen beim <strong>Jobcenter</strong> <st...,de
4,Jobcenter | Landkreis Biberach,https://www.biberach.de/jobcenter,<strong>Jobcenter</strong> Das <strong>Jobcent...,de
...,...,...,...,...
807,Herzlich willkommen beim Jobcenter Wartburgkreis!,https://www.jobcenter-ge.de/Jobcenter/Wartburg...,als Geschäftsführerin des <strong>Jobcenter</s...,de
808,Weimar - Jobcenter Weimar,https://stadt.weimar.de/stadtverwaltung/organi...,Tel.: 03643 4512 970 Fax: 03643 4512 774 Email...,de
809,Jobcenter Weimarer Land,https://www.jobcenter-weimarerland.de/,Bürgergeldbeziehende stellen Anträge ... des <...,de
810,Jobcenter Weimarer Land,https://www.jobcenter-weimarerland.de/,Bürgergeldbeziehende stellen Anträge ... des <...,de


# Function for going to an URL and search for downloadable files inside the domain, the function is made so that it doesn't visit the same page twice

This function was made by ChatGPT4, as the answer for a custom prompt

In [18]:
def find_downloadable_files(url, file_types=None, visited=None, max_depth=3, current_depth=0):
    if file_types is None:
        file_types = ['.pdf','.docx']
    if visited is None:
        visited = set()
    
    # Avoid revisiting the same page
    if url in visited or current_depth > max_depth:
        return []
    visited.add(url)
    
    # Make a request to the URL
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException:
        return []  # Return an empty list in case of request errors

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialize an empty list to store downloadable files
    downloadable_files = []

    # Find all links in the page
    links = soup.find_all('a')

    for link in links:
        href = link.get('href')
        if href:
            full_url = urljoin(url, href)
            parsed_url = urlparse(full_url)
            # Check if the link is within the same domain
            if parsed_url.netloc != urlparse(url).netloc:
                continue  # Skip links from different domains
            # Check if the link is a downloadable file
            if any(full_url.endswith(file_type) for file_type in file_types):
                downloadable_files.append(full_url)
            # Recursive call to follow links
            elif current_depth < max_depth:
                downloadable_files += find_downloadable_files(full_url, file_types, visited, max_depth, current_depth+1)

    return downloadable_files

I am currently organizing all the code, to present it in the repository. I left this script running a whole night and it only got to 30 percent, where it seemed to get stuck. It took 9 hours in total to reach this point.

<img src="images/proof.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

# Finding files

In [21]:
urls = search_results_df['URL']
downloadable_pdfs = []

# Create a tqdm progress bar for the loop
for url in tqdm(urls, desc="Processing URLs", unit="URL"):
    found_files = find_downloadable_files(str(url), ['.pdf'])  # Convert url to string explicitly and pass file_types as list
    pdfs = [file_url for file_url in found_files if file_url.endswith('.pdf')]
    downloadable_pdfs.extend(pdfs)

Processing URLs:   0%|          | 2/812 [02:56<16:37:45, 73.91s/URL] /Users/diegoportillaamarillas/opt/anaconda3/envs/Unstructured10/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Processing URLs:   2%|▏         | 13/812 [12:29<12:47:16, 57.62s/URL]


KeyboardInterrupt: 

# Check if the PDFs are useful by opening them temporarily and checking if there are keywords inside the PDFs

In [25]:
# Function made with the help of Chat GPT4
def contains_keywords(pdf_url, keywords):

    try:
        response = requests.get(pdf_url)
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        with open("temp.pdf", "rb") as f:
            reader = PyPDF2.PdfFileReader(f)
            title = reader.getDocumentInfo().title
            text = ""
            for page_num in range(reader.numPages):
                text += reader.getPage(page_num).extractText()
        if any(keyword.lower() in title.lower() for keyword in keywords):
            return True
        if any(keyword.lower() in text.lower() for keyword in keywords):
            return True
        return False
    
    except Exception as error:
        print(f"Error processing {pdf_url}: {error}")
        return False

In [26]:
keywords = ['Arbeitsmarkt', 'Integrationsprogramm', 'AMIP']

downloadable_pdfs = []

for file_url in found_files:
    if contains_keywords(file_url, keywords):
        downloadable_pdfs.append(file_url)

print(downloadable_pdfs)

Error processing https://www.kreis-calw.de/eGovernment/DS-GVO Führerschein.pdf: PDF starts with '<!doc', but '%PDF-' expected
[]


<img src="images/d_pdfs.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />